**Healthcare Analysis**

*Predicting the Length of Stay for each patient so that the hospitals can optimize resources and function better*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
import warnings
warnings.filterwarnings('ignore')

In [2]:
 #Importing datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

**Data Exploration**

In [3]:
train.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [4]:
train.info()
train.Stay.unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Data columns (total 18 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            318438 non-null  int64  
 1   Hospital_code                      318438 non-null  int64  
 2   Hospital_type_code                 318438 non-null  object 
 3   City_Code_Hospital                 318438 non-null  int64  
 4   Hospital_region_code               318438 non-null  object 
 5   Available Extra Rooms in Hospital  318438 non-null  int64  
 6   Department                         318438 non-null  object 
 7   Ward_Type                          318438 non-null  object 
 8   Ward_Facility_Code                 318438 non-null  object 
 9   Bed Grade                          318325 non-null  float64
 10  patientid                          318438 non-null  int64  
 11  City_Code_Patient                  3139

array(['0-10', '41-50', '31-40', '11-20', '51-60', '21-30', '71-80',
       'More than 100 Days', '81-90', '61-70', '91-100'], dtype=object)

In [5]:
# NA values in train dataset
train.isnull().sum().sort_values(ascending = False)

City_Code_Patient                    4532
Bed Grade                             113
Hospital_code                           0
Admission_Deposit                       0
Age                                     0
Visitors with Patient                   0
Severity of Illness                     0
Type of Admission                       0
patientid                               0
case_id                                 0
Ward_Facility_Code                      0
Ward_Type                               0
Department                              0
Available Extra Rooms in Hospital       0
Hospital_region_code                    0
City_Code_Hospital                      0
Hospital_type_code                      0
Stay                                    0
dtype: int64

In [6]:
# NA values in test dataset
test.isnull().sum().sort_values(ascending = False)

City_Code_Patient                    2157
Bed Grade                              35
case_id                                 0
Age                                     0
Visitors with Patient                   0
Severity of Illness                     0
Type of Admission                       0
patientid                               0
Ward_Facility_Code                      0
Hospital_code                           0
Ward_Type                               0
Department                              0
Available Extra Rooms in Hospital       0
Hospital_region_code                    0
City_Code_Hospital                      0
Hospital_type_code                      0
Admission_Deposit                       0
dtype: int64

In [8]:
#Dimension of train dataset
train.shape

(318438, 18)

In [9]:
#Dimension of test dataset
test.shape

(137057, 17)

In [10]:
# Number of distinct observations in train dataset 
for i in train.columns:
    print(i, ':', train[i].nunique())

case_id : 318438
Hospital_code : 32
Hospital_type_code : 7
City_Code_Hospital : 11
Hospital_region_code : 3
Available Extra Rooms in Hospital : 18
Department : 5
Ward_Type : 6
Ward_Facility_Code : 6
Bed Grade : 4
patientid : 92017
City_Code_Patient : 37
Type of Admission : 3
Severity of Illness : 3
Visitors with Patient : 28
Age : 10
Admission_Deposit : 7300
Stay : 11


In [11]:
# Number of distinct observations in test dataset
for i in test.columns:
    print(i, ':', test[i].nunique())

case_id : 137057
Hospital_code : 32
Hospital_type_code : 7
City_Code_Hospital : 11
Hospital_region_code : 3
Available Extra Rooms in Hospital : 15
Department : 5
Ward_Type : 6
Ward_Facility_Code : 6
Bed Grade : 4
patientid : 39607
City_Code_Patient : 37
Type of Admission : 3
Severity of Illness : 3
Visitors with Patient : 27
Age : 10
Admission_Deposit : 6609


**Data Preparation**

In [13]:
 #replacing missing (NA) values in the Bed Grade column of both the train and test DataFrames. 
train['Bed Grade'].fillna(train['Bed Grade'].mode()[0], inplace = True)
test['Bed Grade'].fillna(test['Bed Grade'].mode()[0], inplace = True)

In [14]:
#replace missing (NA) values in the City_Code_Patient column for both the train and test datasets, by filling the missing values with the mode
train['City_Code_Patient'].fillna(train['City_Code_Patient'].mode()[0], inplace = True)
test['City_Code_Patient'].fillna(test['City_Code_Patient'].mode()[0], inplace = True)

In [15]:
#Label Encoding on the Stay column in the train dataset (convert categorical labels into numeric format)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Stay'] = le.fit_transform(train['Stay'].astype('str'))

In [16]:
train.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,4
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,3
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,4
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,4


In [18]:
#filling the Stay column in the test dataset with a dummy value
test['Stay'] = -1
df = pd.concat([train, test])

In [19]:
df.shape

(455495, 18)

In [20]:
#Label Encoding all the columns in Train and test datasets
for i in ['Hospital_type_code', 'Hospital_region_code', 'Department',
          'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age']:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i].astype(str))

In [21]:
#Seperating both train and test dataset
train = df[df['Stay']!=-1]
test = df[df['Stay']==-1]

**Feature Engineering**

In [23]:
def get_countid_encode(train, test, cols, name):
    # Step 1: Group by `cols` and count `case_id` in train
    temp = train.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
    
    # Step 2: Group by `cols` and count `case_id` in test
    temp2 = test.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
    
    # Step 3: Merge count values into train dataset
    train = pd.merge(train, temp, how='left', on=cols)
    
    # Step 4: Merge count values into test dataset
    test = pd.merge(test, temp2, how='left', on=cols)
    
    # Step 5: Convert the new count columns to float
    train[name] = train[name].astype('float')
    test[name] = test[name].astype('float')
    
    # Step 6: Fill missing values with the median of the counts
    train[name].fillna(np.median(temp[name]), inplace=True)
    test[name].fillna(np.median(temp2[name]), inplace=True)
    
    # Step 7: Return modified train and test DataFrames
    return train, test

In [25]:
#Count case_id by patientid
train, test = get_countid_encode(train, test, ['patientid'], name = 'count_id_patient')
#Count case_id by patientid and Hospital_region_code
train, test = get_countid_encode(train, test, ['patientid', 'Hospital_region_code'], name = 'count_id_patient_hospitalCode')
#Count case_id by patientid and Ward_Facility_Code
train, test = get_countid_encode(train, test, ['patientid', 'Ward_Facility_Code'], name = 'count_id_patient_wardfacilityCode')

In [26]:
# Droping duplicate columns
test1 = test.drop(['Stay', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)
train1 = train.drop(['case_id', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)

In [27]:
# Splitting train data for Naive Bayes
X1 = train1.drop('Stay', axis =1)
y1 = train1['Stay']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size =0.20, random_state =100)

**Models**

**Naive Bayes**

In [28]:
from sklearn.naive_bayes import GaussianNB
target = y_train.values
features = X_train.values
classifier_nb = GaussianNB()
model_nb = classifier_nb.fit(features, target)

*Prediction*

In [29]:
prediction_nb = model_nb.predict(X_test)
from sklearn.metrics import accuracy_score
acc_score_nb = accuracy_score(prediction_nb,y_test)
print("Acurracy:", acc_score_nb*100)

Acurracy: 34.55439015199096


In [33]:
!pip install keras


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
#creating deep learning models
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [35]:
!pip install tensorflow

   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 1.6/390.3 MB 8.4 MB/s eta 0:00:47
   ---------------------------------------- 2.9/390.3 MB 6.7 MB/s eta 0:00:58
   ---------------------------------------- 4.2/390.3 MB 6.4 MB/s eta 0:01:00
    --------------------------------------- 5.2/390.3 MB 6.1 MB/s eta 0:01:03
    --------------------------------------- 6.6/390.3 MB 6.1 MB/s eta 0:01:03
    --------------------------------------- 7.6/390.3 MB 6.0 MB/s eta 0:01:04
    --------------------------------------- 8.9/390.3 MB 5.9 MB/s eta 0:01:05
   - -------------------------------------- 10.2/390.3 MB 6.0 MB/s eta 0:01:04
   - -------------------------------------- 11.3/390.3 MB 5.8 MB/s eta 0:01:06
   - -------------------------------------- 12.3/390.3 MB 5.7 MB/s eta 0:01:07
   - -------------------------------------- 13.9/390.3 MB 5.9 MB/s eta 0:01:05
   - -------------------------------------- 15.2/390.3 MB 5.9 MB/s 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   --------------------- ------------------ 205.0/390.3 MB 4.2 MB/s eta 0:00:44
   --------------------- ------------------ 206.3/390.3 MB 4.2 MB/s eta 0:00:44
   --------------------- ------------------ 206.6/390.3 MB 4.2 MB/s eta 0:00:44
   --------------------- ------------------ 207.1/390.3 MB 4.2 MB/s eta 0:00:44
   --------------------- ------------------ 208.1/390.3 MB 4.2 MB/s eta 0:00:43
   --------------------- ------------------ 209.7/390.3 MB 4.2 MB/s eta 0:00:43
   --------------------- ------------------ 210.8/390.3 MB 4.3 MB/s eta 0:00:43
   --------------------- ------------------ 211.3/390.3 MB 4.3 MB/s eta 0:00:42
   --------------------- ------------------ 212.3/390.3 MB 4.3 MB/s eta 0:00:42
   --------------------- ------------------ 213.4/390.3 MB 4.3 MB/s eta 0:00:42
   ---------------------- ----------------- 214.7/390.3 MB 4.3 MB/s eta 0:00:41
   ---------------------- ----------------- 216.5/390.3 MB 4.3 MB/s eta 0:00:41
   ---------------------- --------------

In [36]:
import tensorflow as tf

In [37]:
 #convert class labels into one-hot encoded vectors
from keras.utils import to_categorical
#Sparse Matrix
a = to_categorical(y_train)
b = to_categorical(y_test)

**Predictions**

**Naive Bayes**

In [39]:
pred_nb = classifier_nb.predict(test1.iloc[:,1:])
result_nb = pd.DataFrame(pred_nb, columns=['Stay'])
result_nb['case_id'] = test1['case_id']
result_nb = result_nb[['case_id', 'Stay']]
result_nb['Stay'] = result_nb['Stay'].replace({0:'0-10', 1: '11-20', 2: '21-30', 3:'31-40', 4: '41-50', 5: '51-60', 6: '61-70', 7: '71-80', 8: '81-90', 9: '91-100', 10: 'More than 100 Days'})

In [40]:
result_nb.head()

,case_id,Stay
0,318439,21-30
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,31-40


**Result**

**Naive Bayes**

In [42]:
print(result_nb.groupby('Stay')['case_id'].nunique())

Stay
0-10                   2598
11-20                 26827
21-30                 72206
31-40                 15639
41-50                   469
51-60                 13651
61-70                    92
71-80                   955
81-90                   296
91-100                    2
More than 100 Days     4322
Name: case_id, dtype: int64
